In [1]:
import os

## switch to the root dir

In [2]:
os.chdir('..')
os.listdir()

['.git',
 '.gitignore',
 '.idea',
 'data_loader',
 'data_processor',
 'jupyter',
 'loggers',
 'main.py',
 'models',
 'original_data',
 'processed_data',
 'README.md',
 'saved_models',
 'trainers']

In [3]:
from data_processor.utils import load_numpy_arrays, numpy_to_tensor
from models.CNN import CNN_2d_block
from models.MLP import MLP
import trainers.SGD_trainer
from loggers.statistics_loggers import plot_numerical_arrays
import torch
from torch import nn


## Load preprocessed data

In [4]:
X_train, y_train, X_val, y_val = load_numpy_arrays(['data_train_X.npy','data_train_y.npy', 'data_val_X.npy','data_val_y.npy']
                  , path_prefix='processed_data/processed_waves/4-genres/')

In [5]:
X_train, y_train, X_val, y_val = numpy_to_tensor([X_train, y_train, X_val, y_val])

In [6]:
y_train = y_train.type(torch.int64)
y_val = y_val.type(torch.int64)

In [7]:
print(X_train.shape)
print(y_train.shape)

torch.Size([640, 501, 20])
torch.Size([640])


In [8]:
print(X_train[0])
print(y_train[0])

tensor([[-2.6793, -3.5519, -4.7389,  ..., -3.0967, -3.5594, -3.4670],
        [-3.1634, -3.8503, -4.6526,  ..., -2.4343, -2.6496, -2.7823],
        [-2.8878, -3.4747, -4.8224,  ..., -2.5756, -2.5007, -2.7433],
        ...,
        [-0.2473,  0.0616,  0.4944,  ..., -0.8849, -1.0255, -0.9458],
        [-0.1621,  0.0802,  0.1030,  ..., -0.9531, -0.9674, -0.9041],
        [-0.2025, -0.1149,  0.0338,  ..., -0.2389, -0.2379, -0.1873]])
tensor(1)


In [9]:
print(X_val.shape)
print(y_val.shape)

torch.Size([80, 501, 20])
torch.Size([80])


## Load model

1. define model under the folder models
2. import and define the model

In [53]:
class simple_CRDNN(nn.Module):
    def __init__(self):
        super(simple_CRDNN, self).__init__()
        
        '''
        self.conv1 = CNN_2d_block(
            in_channels = 1,
            out_channels = 4,
            kernel_size = (5, 5),
            stride = (2, 2),
            padding = (2, 2),
            pooling = 'avg',
            pooling_kernel_size = (2, 2),
            pooling_stride = (2, 2),
            pooling_padding = 0,
            activation = 'ReLU',
            batch_norm = True 
            
        )
        
        self.conv2 = CNN_2d_block(
            in_channels = 4,
            out_channels = 8,
            kernel_size = (5, 5),
            stride = (1, 1),
            padding = (2, 2),
            pooling = 'max',
            pooling_kernel_size = (2, 2),
            pooling_stride = (2, 2),
            pooling_padding = 0,
            activation = 'ReLU',
            batch_norm = True 
            
        )
        '''
        
        self.LSTM = nn.LSTM(input_size=20, hidden_size=40, num_layers=8, dropout=0.15, batch_first=True)
        self.MLP = MLP(40, 4, [100])
        
        
    
    def forward(self, x):
        out = x
        # out = self.conv1(x)
        # out = self.conv2(out)
        # out = torch.flatten(torch.transpose(out, 1, 2), 2)
        out, _ = self.LSTM(out)
        out = out[:, -1, :]
        out = self.MLP(out)
        return out

In [54]:
# eg. MLP
model = simple_CRDNN()

In [55]:
n_feat = X_train.shape[2]
n_time = X_train.shape[1]

In [57]:
X_a = X_train[:10]#.reshape((-1, 1, n_time, n_feat))
pred = model(X_a)

## Define loss and optimizer

In [46]:
lr = 0.05
batch_size = 10
num_epoch = 10
model_name = 'CRNN'
saved_model_name = 'saved_models/saved_' + model_name + '_wave.pt'

In [47]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

## Train

use trainers.SGD_trainer.train or define a trainer

parameters of SGD_trainer.train
- model
- train_array: a tuple (X_train, y_train, X_val, y_val)
- loss
- optimizer
- batch_size
- num_epoch
- device

In [58]:
X_train_2d = X_train.reshape((-1, 1, n_time, n_feat))
print(X_train.shape)
X_val_2d = X_val.reshape((-1, 1, n_time, n_feat))
training_loss_array, training_accuracy_array, validation_loss_array, validation_accuracy_array = trainers.SGD_trainer.train(model, (X_train, y_train, X_val, y_val), 
                                                                                                                            loss, optimizer, batch_size=batch_size, num_epoch=num_epoch, device='cuda')

torch.Size([640, 501, 20])


RuntimeError: CUDA out of memory. Tried to allocate 2.26 GiB (GPU 0; 4.00 GiB total capacity; 504.13 MiB already allocated; 650.48 MiB free; 880.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

## Plot

plot_numerical_arrays: plot multiple arrays with the same length

parameters:
- num_arrays: numerical arrays with the same length
- labels: labels of each array(with the same order of num_arrays)
- xlabel
- ylabel
- title

In [ ]:
plot_numerical_arrays([training_loss_array, validation_loss_array], ['training loss', 'validation loss'], 
                      xlabel='batches', ylabel='loss', title='train and validation losses for logistic regression')

In [ ]:
plot_numerical_arrays([training_accuracy_array, validation_accuracy_array], ['training accuracy', 'validation accuracy'], 
                      xlabel='batches', ylabel='accuracy', title='train and validation accuracies for logistic regression')

## Save model

Save model in 'saved_models/saved_modelname_wave.pt'

In [ ]:
torch.save(model.state_dict(), saved_model_name)